In [2]:
import streamlit as st
import numpy as np
import plotly.graph_objects as go
import pandas as pd

In [5]:
df = pd.read_csv("Export_TER_juin2023_FIX_SafwanChendeb.csv",sep=";")
print(df.info)

<bound method DataFrame.info of          N° ENREG Code CAF  ASS_FRQ_KHZ  ASS_LGBD_KHZ  TER_ANT_AZM_MAX  \
0       1028247.0        K     38577000       14000.0             90.0   
1       1033878.0        K     19376875       13700.0            204.5   
2       1428858.0        K     26145000       56000.0            320.7   
3       1502495.0        K      6920000       40000.0            307.4   
4        339609.0        K     38552500        7000.0            319.9   
...           ...      ...          ...           ...              ...   
823645  1419579.0        K     22128750       28000.0            195.1   
823646   124559.0        J         3398           0.0              NaN   
823647  1111112.0        K     23241750       14000.0            182.8   
823648  1508178.0        K     26173000      112000.0            156.6   
823649  1276551.0        K     38570000       56000.0            168.4   

        TER_ANT_ANG  PT_LONG_DEG  PT_LONG_MIN  PT_LONG_SEC PT_LONG_ORIENT  \
0 

In [12]:
df.isnull().sum()

N° ENREG             105
Code CAF               0
ASS_FRQ_KHZ            0
ASS_LGBD_KHZ        1470
TER_ANT_AZM_MAX    23791
TER_ANT_ANG        23788
PT_LONG_DEG            0
PT_LONG_MIN            0
PT_LONG_SEC            0
PT_LONG_ORIENT         0
PT_LAT_DEG             0
PT_LAT_MIN             0
PT_LAT_SEC             0
PT_LAT_ORIENT          0
BASE                   0
MVT_CODE               0
jour                   0
mois                   0
année                  0
dtype: int64

In [9]:
df[['jour', 'mois', 'année']] = df['Date CAF'].str.split('/', expand=True)

In [11]:
df = df.drop(columns=['Date CAF'])